In [140]:
from datetime import datetime
from transformation_handler import readData
import pandas as pd
import numpy as np

In [ ]:
dfs = readData(limit=1000000)
dfs.keys()

In [155]:
austin_in = dfs['shelter_austin_intakes']
austin_out = dfs['shelter_austin_outcomes']
austinn_in = austin_in.copy()
austinn_out = austin_out.copy()

In [158]:
austinn_in.rename(columns={'animal_type':'type','sex_upon_outcome':'sex','datetime':'intake_date'},inplace=True)
austinn_out.rename(columns={'animal_type':'type','datetime':'outcome_date','sex_upon_outcome':'sex'},inplace=True)
austinn_in.sort_values(by='intake_date', inplace=True)
austinn_out.sort_values(by= 'outcome_date', inplace=True)
austinn_in['intake_date'] = pd.to_datetime(austinn_in['intake_date'])
date_columns = ['date_of_birth','outcome_date']
for col in date_columns:
    austinn_out[col] = pd.to_datetime(austinn_out[col])
austinn_in.drop_duplicates(subset='animal_id', keep='first', inplace=True)
austinn_out.drop_duplicates(subset='animal_id', keep='first', inplace=True)
combined_df = pd.merge(austinn_in, austinn_out, on='animal_id', how = 'left')
combined_df['outcome_type'] = combined_df['outcome_type'].fillna('Pending')
combined_df['sex'].replace({'NULL': 'Unknown', 'Intact Male': 'Male', 'Intact Female': 'Female', 'Neutered Male': 'Neutered', 'Spayed Female': 'Spayed'}, inplace=True)
combined_df['type'] = np.where(~combined_df['type'].isin(['Cat', 'Dog']), 'Other', combined_df['type'])
combined_df['days_in_shelter'] = (combined_df['outcome_date'] - combined_df['intake_date']).dt.days
combined_df['animal_id'] = combined_df['animal_id'].apply(lambda x: f'AUS-{x}')
combined_df['region'] = 'Austin'
combined_df.outcome_type.replace({'Relocate':'Transfer', 'Rto-Adopt': 'Return to Owner', 'Euthanasia':'Euthanize'},inplace=True)

In [159]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140033 entries, 0 to 140032
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   animal_id        140033 non-null  object        
 1   type             140033 non-null  object        
 2   breed            140033 non-null  object        
 3   color            140033 non-null  object        
 4   intake_date      140033 non-null  datetime64[ns]
 5   intake_type      140033 non-null  object        
 6   sex              139173 non-null  object        
 7   date_of_birth    139173 non-null  datetime64[ns]
 8   outcome_date     139173 non-null  datetime64[ns]
 9   outcome_type     140033 non-null  object        
 10  days_in_shelter  139173 non-null  float64       
 11  region           140033 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(8)
memory usage: 13.9+ MB


In [145]:
combined_df.type.value_counts()

Dog      73172
Cat      57880
Other     8981
Name: type, dtype: int64

In [146]:
combined_df.breed.value_counts()

Domestic Shorthair Mix                                 31544
Domestic Shorthair                                     16440
Pit Bull Mix                                            7360
Labrador Retriever Mix                                  6836
Chihuahua Shorthair Mix                                 6013
                                                       ...  
Siberian Husky/Rottweiler                                  1
American Eskimo/Alaskan Husky                              1
American Pit Bull Terrier/American Pit Bull Terrier        1
Great Pyrenees/Queensland Heeler                           1
American Staffordshire Terrier/Rottweiler                  1
Name: breed, Length: 2874, dtype: int64

In [147]:
combined_df.color.value_counts(ascending=True)

Black Brindle/Gray               1
Liver/Buff                       1
Calico/Gray Tabby                1
Orange Tabby/Tortie Point        1
Silver Tabby/Black               1
                             ...  
White                         4857
Brown                         6065
Brown Tabby                   8877
Black                        12112
Black/White                  14338
Name: color, Length: 633, dtype: int64

In [148]:
combined_df.sex.value_counts()

Neutered    45910
Spayed      42544
Male        19390
Female      19208
Unknown     12121
Name: sex, dtype: int64

In [149]:
combined_df.intake_type.value_counts()

Stray                 100984
Owner Surrender        23863
Public Assist           7747
Wildlife                5951
Abandoned               1245
Euthanasia Request       243
Name: intake_type, dtype: int64

In [160]:
combined_df.outcome_type.value_counts()

Adoption           64679
Transfer           42226
Return to Owner    20438
Euthanize           9559
Died                1420
Pending              885
Disposal             747
Missing               76
Stolen                 3
Name: outcome_type, dtype: int64